In [40]:
import os
import cv2
import csv
import keras
import pickle
import h5py
import sklearn
import tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Lambda
from keras.backend import tf as ktf
from keras.optimizers import SGD
from keras import optimizers
print ('Loaded')

Loaded


In [41]:
# This cell is the COPY of the above cell. 
Images = []
steering_angles = []
counter = []
with open('data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        Images.append('data/data/IMG/'+line[0].split('/')[-1])
        Images.append('data/data/IMG/'+line[1].split('/')[-1])
        Images.append('data/data/IMG/'+line[2].split('/')[-1])
        steering_angle = float(line[3])
        steering_angles.append(steering_angle)
        steering_angles.append(steering_angle+0.2)
        steering_angles.append(steering_angle-0.2)

        counter.append(1)
        counter.append(1)
        counter.append(1)
        
        if(steering_angle >0.2 or steering_angle<(-0.2)):
            Images.append('data/data/IMG/'+line[0].split('/')[-1])
            steering_angles.append(-steering_angle)
            counter.append(0)
            
samples = list(zip(Images, steering_angles, counter))
print(len(samples))

53518


In [42]:
from sklearn.model_selection import train_test_split
train_validation, test = train_test_split(samples, test_size=0.1)
train_samples, validation_samples = train_test_split(train_validation, test_size=0.2)


In [43]:
import sklearn
import cv2
def generator(samples, batch_size=32):
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                counter = batch_sample[2]
                
                if(counter):
                    center_image = cv2.imread(batch_sample[0])
                    center_angle = batch_sample[1]
                    center_image = cv2.resize(center_image, (200,66))
                    center_image = cv2.cvtColor(center_image, cv2.COLOR_RGB2GRAY)
                    center_image = np.reshape(center_image,(66,200,1))
                else:
                    center_image = cv2.imread(batch_sample[0])
                    center_angle = batch_sample[1]
                    center_image = cv2.resize(center_image, (200,66))
                    center_image = cv2.cvtColor(center_image, cv2.COLOR_RGB2GRAY)
                    center_image = cv2.flip(center_image,1)
                    center_image = np.reshape(center_image,(66,200,1))
                    
                images.append(center_image)
                angles.append(center_angle)
                
            X_train = np.array(images)
            y_train = np.array(angles)
            
            yield sklearn.utils.shuffle(X_train, y_train)
            


In [45]:
model = Sequential()
model.add(keras.layers.convolutional.Cropping2D(cropping=((20,9),(0,0)), input_shape = (66,200,1)))
model.add( Lambda(lambda x: (x/255.0) - 0.5))

model.add(Conv2D(24, 5, 5, border_mode="same", init='glorot_uniform',subsample=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(36, 5, 5, border_mode="same", init='glorot_uniform' ,subsample=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(48, 5, 5, border_mode="same", init='glorot_uniform',subsample=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(64, 3, 3,border_mode="same", init='glorot_uniform',subsample=(1,1)))
model.add(Activation('relu'))

model.add(Conv2D(64, 3, 3,border_mode="same", init='glorot_uniform',subsample=(1,1)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1164, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dense(100, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dense(50, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dense(10, init='glorot_uniform'))
model.add(Activation('relu'))
model.add(Dense(1, init='glorot_uniform'))


In [46]:
train_generator      = generator( train_samples, batch_size=32 )
validation_generator = generator( validation_samples, batch_size=32 )

In [47]:
model.compile(loss = 'mean_squared_error', optimizer='adam')

model.fit_generator (train_generator, samples_per_epoch = len(train_samples), validation_data = validation_generator, nb_val_samples = len(validation_samples), nb_epoch = 10)
model.save('final1.h5')

Epoch 1/10
38532/38532 [==============================] - 70s - loss: 0.0324 - val_loss: 0.0284
Epoch 2/10
38532/38532 [==============================] - 69s - loss: 0.0259 - val_loss: 0.0268
Epoch 3/10
38532/38532 [==============================] - 69s - loss: 0.0228 - val_loss: 0.0247
Epoch 4/10
38532/38532 [==============================] - 69s - loss: 0.0207 - val_loss: 0.0244
Epoch 5/10
38532/38532 [==============================] - 69s - loss: 0.0191 - val_loss: 0.0237
Epoch 6/10
38532/38532 [==============================] - 69s - loss: 0.0173 - val_loss: 0.0244
Epoch 7/10
38532/38532 [==============================] - 69s - loss: 0.0164 - val_loss: 0.0224
Epoch 8/10
38532/38532 [==============================] - 69s - loss: 0.0151 - val_loss: 0.0222
Epoch 9/10
38532/38532 [==============================] - 69s - loss: 0.0137 - val_loss: 0.0230
Epoch 10/10
38532/38532 [==============================] - 69s - loss: 0.0128 - val_loss: 0.0212


In [48]:
test_generator = generator(test, batch_size = 32)
value = model.evaluate_generator(test_generator, val_samples = len(test))


In [49]:
print("The test loss value is: ",value)

The test loss value is:  0.0199910807235
